Section 1 : Import Libraries

In [20]:
import requests # to fetch data from api
import pandas as pd #for handling and analysing dat
import numpy as np # for numerical operations
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder # to convert catogerical data into numericals values
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.metrics import accuracy_score ,mean_squared_error
from datetime import datetime, timedelta
import pytz

In [21]:
API_KEY = '49d46a7e49919df6164bf7109a9bd3a6'
BASE_URL = 'https://api.openweathermap.org/data/2.5/' # for making API request

Section 2 : Fetch current Weather Data

In [22]:
def get_current_weather(city):
    url = f'{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric' # construct the API request URL
    response = requests.get(url)  # sending GET request to the API endpoint
    data = response.json()
    return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'pressure' : data['main']['pressure'],
      'wind_Gust_Speed': (data['wind']['speed']),
    }

2. Read Historical Data

In [23]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

3. Prepare data for Training

In [36]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data[['MinTemp' , 'MaxTemp' , 'WindGustDir' , 'WindGustSpeed' , 'Humidity' , 'Pressure' , 'Temp']]
  y= data['RainTomorrow']

  return X ,y , le

4. Train Rain Prediction Model

In [37]:
def train_rain_model(X , y):
  X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)
  model = RandomForestClassifier(n_estimators=100 , random_state=42)
  model.fit(X_train , y_train)
  y_pred = model.predict(X_test)

  print("Mean Squarred Error for Rain Model")
  print(mean_squared_error(y_test, y_pred))

  return model

5. Prepare regression data

In [43]:
def prepare_regression_data(data, feature):
  X , y = [] , []
  for i in range (len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i + 1])

  X= np.array(X).reshape(-1, 1)
  y = np.array(y)
  return X, y

Train Regression Model

In [39]:
def train_regression_model(X , y):
  model = RandomForestRegressor(n_estimators=100 , random_state=42)
  model.fit(X , y)
  return model

Predict Future

In [40]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array(predictions[-1]).reshape(-1, 1))
    predictions.append(next_value[0])

  return predictions[1:]

Weather Analysis Function

In [48]:
def weather_view():
  city = input ('Enter the city name : ')
  current_weather = get_current_weather(city)

  historical_data = read_historical_data('weather.csv')
  X , y , le = prepare_data(historical_data)

  rain_model = train_rain_model( X,y)
# Map wind direction to compass points

  wind_deg = current_weather ['wind_gust_dir']  % 360

  compass_points = [
      ("Ν", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
       ("ENE", 56.25, 78.75), ("E", 78.75, 181.25), ("ESE", 101.25, 123.75),
        ("SE", 121.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
         ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
          ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
           ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start < wind_deg < end)
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_Gust_Speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }
  current_df = pd.DataFrame([current_data])
  # Rain Prediction
  rain_prediction = rain_model.predict(current_df)[0]

  # prepare regression model for temperature and humidity

  x_temp, y_temp = prepare_regression_data(historical_data , 'Temp')

  x_hum , y_hum = prepare_regression_data(historical_data , 'Humidity')

  temp_model = train_regression_model(x_temp , y_temp)

  hum_model = train_regression_model(x_hum , y_hum)

  #Predict future temp and humidity

  future_temp = predict_future(temp_model , current_weather['temp_min'])
  future_humidity = predict_future(hum_model , current_weather['humidity'])

  # prepare time for future prediction

  timezone = pytz.timezone('Asia/Bangkok')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0 , second=0 , microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #display Results

  print(f"city : {city}, {current_weather['country']}")
  print(f"Current Temperature : {current_weather['current_temp']}")
  print(f"Feels like : {current_weather['feels_like']}")
  print(f"Minimum Temperature : {current_weather['temp_min']}°C")
  print(f"Maximum Temperature : {current_weather['temp_max']}°C")
  print(f"Humidity : {current_weather['humidity']}%")
  print(f"Weather Prediction : {current_weather['description']}")
  print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions :")

  for time, temp in zip(future_times, future_temp):
    print(f"{time} : {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions :")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time} : {round(humidity, 1)}°C")
weather_view()

Enter the city name : Ahmedabad
Mean Squarred Error for Rain Model
0.1506849315068493
city : Ahmedabad, IN
Current Temperature : 28
Feels like : 32
Minimum Temperature : 28°C
Maximum Temperature : 28°C
Humidity : 78%
Weather Prediction : mist
Rain Prediction : Yes

Future Temperature Predictions :
13:00 : 27.7°C
14:00 : 22.8°C
15:00 : 25.4°C
16:00 : 24.7°C
17:00 : 23.4°C

Future Humidity Predictions :
13:00 : 48.6°C
14:00 : 48.6°C
15:00 : 48.6°C
16:00 : 48.6°C
17:00 : 48.6°C
